# ML классификатор на основе нейронной сети

In [161]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import pandas as pd

In [162]:
# Загружаем файл с разделителем ';' в DF
df = pd.read_csv(fr'nn_features_and_target.csv', delimiter=';')
# df.columns = df.columns.astype(str)
# df['29'] = df.apply(lambda x: 1.0 if x['28'] == 0.0 else 0.0, axis=1)
print(df.to_string(max_rows=8, max_cols=30))
print()
print(df.target.value_counts(normalize=True))

       TRADEDATE SHORTNAME next_name next_up/down  next_size_body         3         4         5         6         7         8         9        10        11        12  ...  24  25  26  27  28  29  30  F_0  F_1  F_2  F_3  F_4  F_5  F_6  target
0     2015-02-02  RTS-3.15  RTS-3.15           up          4210.0  1.000000  1.000000  0.958552  0.958513  0.909569  0.906579  0.825061  0.786700  0.663511  0.645015  ...   1   0   0   0   0   0   0    1    1    0    1    1    1    0     5.0
1     2015-02-03  RTS-3.15  RTS-3.15         down          1720.0  1.000000  0.999952  0.938883  0.935652  0.831637  0.783216  0.635221  0.612335  0.343496  0.112769  ...   0   1   0   0   0   0   0    1    1    1    1    1    1    0     1.0
2     2015-02-04  RTS-3.15  RTS-3.15           up          4200.0  1.000000  0.950573  0.950530  0.902611  0.899701  0.808305  0.764537  0.630159  0.609508  0.362685  ...   0   0   1   0   0   0   0    0    0    1    1    0    0    1     5.0
3     2015-02-05  RTS-3.15  RTS-

In [163]:
# Указание процента тестового набора (например, 20%)
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42, shuffle=False)

print(df_train.to_string(max_rows=8, max_cols=30))
print(df_train.shape)
print()
print(df_test.to_string(max_rows=8, max_cols=30))
print(df_test.shape)

       TRADEDATE  SHORTNAME  next_name next_up/down  next_size_body    3         4         5         6         7         8         9        10        11        12  ...  24  25  26  27  28  29  30  F_0  F_1  F_2  F_3  F_4  F_5  F_6  target
0     2015-02-02   RTS-3.15   RTS-3.15           up          4210.0  1.0  1.000000  0.958552  0.958513  0.909569  0.906579  0.825061  0.786700  0.663511  0.645015  ...   1   0   0   0   0   0   0    1    1    0    1    1    1    0     5.0
1     2015-02-03   RTS-3.15   RTS-3.15         down          1720.0  1.0  0.999952  0.938883  0.935652  0.831637  0.783216  0.635221  0.612335  0.343496  0.112769  ...   0   1   0   0   0   0   0    1    1    1    1    1    1    0     1.0
2     2015-02-04   RTS-3.15   RTS-3.15           up          4200.0  1.0  0.950573  0.950530  0.902611  0.899701  0.808305  0.764537  0.630159  0.609508  0.362685  ...   0   0   1   0   0   0   0    0    0    1    1    0    0    1     5.0
3     2015-02-05   RTS-3.15   RTS-3.15      

In [164]:
# # Разделяем датафрейм на признаки (X) и целевую переменную (y)
# X = df.drop('28', axis=1).values
# y = df['28'].values
# print(y)

X_train = df_train.iloc[:, 5:-1]
y_train = df_train.iloc[:, -1]

X_test = df_test.iloc[:, 5:-1]
y_test = df_test.iloc[:, -1]

In [165]:
# # Преобразуем категориальный таргет в числовой формат
# label_encoder = LabelEncoder()
# y_encoded = label_encoder.fit_transform(y)
# print(y_encoded)

In [166]:
# Преобразуем числовые метки в one-hot encoding
# y_categorical = to_categorical(y)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
# print(y_categorical)
print(y_train)

[[0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 ...
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]]


In [167]:
# # Разделяем данные на тренировочный и тестовый наборы
# X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42, shuffle=False)
# # print(y_test)

In [168]:
# Создаем модель нейронной сети
model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))  # Входной слой
for _ in range(2):
    model.add(Dense(512, activation='relu', ))
    model.add(BatchNormalization())
    model.add(Dropout(0.80))
model.add(Dense(y_train.shape[1], activation='softmax'))  # Выходной слой

In [169]:
# Компилируем модель по параметрам
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [170]:
# Определение функции обратного вызова EarlyStopping
early_stopping = EarlyStopping(
    # monitor='val_loss',  # Мониторим валидационную потерю
    monitor='val_accuracy',  # Мониторинг точности на валидационном наборе
    patience=500,             # Количество эпох без улучшения до остановки
    verbose=1,               # Вывод информации в процессе обучения
    restore_best_weights=True  # Восстанавливаем веса модели
)

# Определение функции обратного вызова ModelCheckpoint
model_checkpoint = ModelCheckpoint(
    filepath='best_model_pred.keras',   # Путь для сохранения модели
    # monitor='val_loss',         # Мониторим валидационную потерю
    # mode='min',
    monitor='val_accuracy',     # Мониторинг точности на валидационном наборе
    mode='max',
    save_best_only=True,        # Сохраняем только лучшую модель
    verbose=1                   # Вывод информации в процессе сохранения
)

# Обучение модели с использованием функций обратного вызова
history = model.fit(
    X_train, y_train,
    epochs=100,                 # Максимальное количество эпох
    batch_size=32,              # Размер мини-пакета
    validation_split=0.2,       # Доля данных для валидации
    callbacks=[early_stopping, model_checkpoint]  # Список функций обратного вызова
)

Epoch 1/100
38/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1540 - loss: 4.3726   
Epoch 1: val_accuracy improved from -inf to 0.15426, saving model to best_model_pred.keras
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.1554 - loss: 4.3398 - val_accuracy: 0.1543 - val_loss: 1.7832
Epoch 2/100
38/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1747 - loss: 3.5723 
Epoch 2: val_accuracy improved from 0.15426 to 0.16223, saving model to best_model_pred.keras
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1720 - loss: 3.5898 - val_accuracy: 0.1622 - val_loss: 1.7973
Epoch 3/100
36/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2004 - loss: 3.2451 
Epoch 3: val_accuracy did not improve from 0.16223
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2008 - loss: 3.2530 - val_accuracy: 0.1436 - val_loss: 1.8512
Epoch 4/100
37/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2033 - loss: 3.0823 
Epoch 4: val_accuracy did not improve from 0.16223
47/47 ━━━━━━━━━━━━━━━━

In [171]:
# Обучаем модель
# model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

In [172]:
# Оцениваем модель на тестовом наборе
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy}')

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 714us/step - accuracy: 0.2509 - loss: 1.7525
Test accuracy: 0.25531914830207825


In [173]:
predictions = model.predict(X_test)

# Преобразование чисел в строки с фиксированным количеством десятичных знаков
formatted_arr = np.array([[f'{num:.6f}' for num in row] for row in predictions])

# Преобразование строк обратно в числа
formatted_num_arr = np.array([[float(num) for num in row] for row in formatted_arr])

# Преобразуем массив в DataFrame
df = pd.DataFrame(formatted_num_arr)

# Добавление столбца предсказанной категории
# df['predict'] = df.apply(lambda x: 'down' if x[0] > x[1] else 'up', axis=1)

# df['ind_max'] = df.apply(lambda x: x.idxmax(axis=1))
df['ind_max'] = df.idxmax(axis=1)

# Вывод вероятностей для примера из тестовой выборки
# print(df)
print(df.to_string(max_rows=20, max_cols=10))

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
            0         1         2         3         4         5  ind_max
0    0.158844  0.236452  0.148263  0.105536  0.223533  0.127372        1
1    0.175810  0.207550  0.106235  0.069833  0.275134  0.165438        4
2    0.191324  0.245838  0.104599  0.086918  0.246424  0.124897        4
3    0.216469  0.215929  0.090652  0.106850  0.244881  0.125218        4
4    0.105218  0.223613  0.126531  0.094060  0.230476  0.220102        4
5    0.251848  0.170807  0.069344  0.107818  0.200632  0.199550        0
6    0.267269  0.156002  0.048359  0.061697  0.222352  0.244320        0
7    0.191772  0.258250  0.076970  0.103992  0.248292  0.120724        1
8    0.196303  0.242046  0.112156  0.110821  0.227414  0.111258        1
9    0.096030  0.218746  0.153506  0.193877  0.241705  0.096135        4
..        ...       ...       ...       ...       ...       ...      ...
460  0.133235  0.198532  0.139389  0.167906  0.221237  0.139700        4
461  0.1192

In [174]:
print(df.ind_max.value_counts(normalize=True))
print()
print(df.ind_max.value_counts())

ind_max
4    0.785106
1    0.200000
0    0.012766
3    0.002128
Name: proportion, dtype: float64

ind_max
4    369
1     94
0      6
3      1
Name: count, dtype: int64


In [175]:
# Преобразуем числовые метки обратно в категориальные 
y_test_lst = [('down' if y[0] == 1.0 else 'up') for y in y_test]

# Добавление колонки с реальными напрвлениями баров
df['real'] = y_test_lst
print(df)

            0         1         2         3         4         5  ind_max  real
0    0.158844  0.236452  0.148263  0.105536  0.223533  0.127372        1  down
1    0.175810  0.207550  0.106235  0.069833  0.275134  0.165438        4  down
2    0.191324  0.245838  0.104599  0.086918  0.246424  0.124897        4    up
3    0.216469  0.215929  0.090652  0.106850  0.244881  0.125218        4    up
4    0.105218  0.223613  0.126531  0.094060  0.230476  0.220102        4  down
..        ...       ...       ...       ...       ...       ...      ...   ...
465  0.276798  0.175983  0.049173  0.073327  0.196924  0.227794        0    up
466  0.144568  0.193209  0.079418  0.060122  0.327374  0.195309        4    up
467  0.144677  0.263772  0.113821  0.072741  0.282258  0.122732        4    up
468  0.159933  0.221734  0.131894  0.117697  0.241245  0.127496        4    up
469  0.122219  0.198372  0.104813  0.073585  0.283572  0.217438        4  down

[470 rows x 8 columns]


In [176]:
# Подсчет количества совпадений
num_matches = (df['predict'] == df['real']).sum()
print(num_matches)

# Подсчет количества не совпадений
num_no_matches = (df['predict'] != df['real']).sum()
print(num_no_matches)

KeyError: 'predict'